In [1]:
import boto3, os
import pandas as pd
import json
os.environ['AWS_DEFAULT_REGION']="us-east-1"
client = boto3.client('lex-models')

## Prepare slot values from dataset

In [7]:
movies = pd.read_csv('./movies.csv',  names=['ITEM_ID','title','genre'], engine= 'python', header = 0)
movies.head()

,ITEM_ID,title,genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
import re

def split_it(title):
    p = re.compile('(.*)(\(\d\d\d\d\))')
    if p.search(title):
        return p.search(title).group(1)
    else: 
        return title

titles =movies['title'].apply(split_it).unique()

In [16]:
slot_value_list = []

for val in titles[:3000].tolist():
    data = {}
    data['value'] = val
    slot_value_list.append(data)



## Create slot type for watched movies

In [22]:
#checksum = client.get_slot_type(name='watchedMovie', version='$LATEST')['checksum']
response = client.put_slot_type(
    name='watchedMovie',
    description='A movie which the user has already watched',
    valueSelectionStrategy='ORIGINAL_VALUE',
    enumerationValues=slot_value_list,
    createVersion=False
)

## Create movie recommendation intent

In [26]:

response = client.put_intent(
    name='MovieRecommenderIntent',
    description='This intent allows to get movie recommendations',
    slots=[
        {
            'name': 'watchedMovie',
            'description': 'A movie that the user already watched',
            'slotConstraint': 'Required',
            'slotType': 'watchedMovie',
            'slotTypeVersion': '$LATEST',
            'valueElicitationPrompt': {
                'messages': [
                    {
                        'contentType': 'PlainText',
                        'content': 'To make a good recommendation. Tell me a movie which you really liked!'
                    },
                ],
                'maxAttempts': 3
            },
            'priority': 1
        },
    ],
    sampleUtterances=[
        'Can you recommend me a movie','Can you recommend me a movie that is similar to {watchedMovie}', 'Which movie should i watch', 'Which movie should i see', 'What is a great movie', "Tell me a great movie similar to {watchedMovie}"
    ],
    confirmationPrompt={
        'messages': [
            {
                'contentType': 'PlainText',
                'content': 'Do you want a movie similar to {watchedMovie}'

            }
        ],
        'maxAttempts': 1
    },
    rejectionStatement={
        'messages': [
            {
                'contentType': 'PlainText',
                'content': 'Okay, not recommending you a movie!'

            },
        ],
        'responseCard': 'string'
    },
    fulfillmentActivity={
        'type': 'CodeHook',
        'codeHook': {
            'messageVersion' : '1.0',
            'uri': 'arn:aws:lambda:us-east-1:028626156119:function:movie-chatbot-movieChatbotFunction-SOCS3XYBBQGH'
        }
    },
        createVersion=False
)

## Create the chatbot

In [27]:
response = client.put_bot(
    name='MovieRecommendationBot',
    description='This bot provides movie recommendations based movies you have seen',
    intents=[
        {
            'intentName': 'MovieRecommenderIntent',
            'intentVersion': '$LATEST'
        },
    ],
    clarificationPrompt={
        'messages': [
            {
                'contentType': 'PlainText',
                'content': 'Hi. If you want a movie recommendation, ask What is a great movie?'
                
            },
        ],
        'maxAttempts': 3
    },
    abortStatement={
        'messages': [
            {
                'contentType': 'PlainText',
                'content': 'Thanks for using the movie recommender bot. Goodbye!'
                
            },
        ]
    },
    idleSessionTTLInSeconds=120,
    voiceId='Joanna',
    processBehavior='BUILD',
    locale='en-US',
    childDirected=False,
    createVersion=False
)

## Delete resources

In [18]:
response = client.delete_bot(
    name='MovieRecommendationBot'
)


In [20]:
response = client.delete_intent(
    name='MovieRecommenderIntent'
)

In [21]:
response = client.delete_slot_type(
    name='watchedMovie'
)

## Personalize test

In [10]:
client = boto3.client('personalize-runtime')
s3 = boto3.client('s3')

In [111]:
movieName = "Silence of the lambs"

In [112]:
s3.download_file('movie-chatbot-deployment', 'movies.dat', '/tmp/movies.dat')
movies = pd.read_csv('/tmp/movies.dat', sep='::', names=['ITEM_ID','title','genre'], engine= 'python')

def getIDForMovieName(movieName):
    filteredMovies = movies[movies['title'].str.contains(movieName, case=False)]
    if (len(filteredMovies.index) > 0 ):
        print(filteredMovies)
        return filteredMovies.iloc[0][0]
    return None

In [113]:
itemId = getIDForMovieName(movieName)

     ITEM_ID                             title           genre
589      593  Silence of the Lambs, The (1991)  Drama|Thriller


In [119]:
rec_response = client.get_recommendations(
    campaignArn='arn:aws:personalize:us-east-1:028626156119:campaign/DEMO-sims-campaign-42079',
    itemId=str(itemId),
    numResults=10
)
rec_items = [int(x['itemId']) for x in rec_response['itemList']]
movies.loc[rec_items[:5]].values.tolist()

[[1379, 'Young Guns II (1990)', 'Action|Comedy|Western'],
 [321, 'Strawberry and Chocolate (Fresa y chocolate) (1993)', 'Drama'],
 [1232, 'Stalker (1979)', 'Mystery|Sci-Fi'],
 [612, 'Pallbearer, The (1996)', 'Comedy'],
 [1105, 'Children of the Corn IV: The Gathering (1996)', 'Horror']]

In [120]:
rec_response = client.get_recommendations(
    campaignArn='arn:aws:personalize:us-east-1:028626156119:campaign/DEMO-sims-hpo-campaign-42079',
    itemId=str(itemId),
    numResults=10
)
rec_items = [int(x['itemId']) for x in rec_response['itemList']]
movies.loc[rec_items[:10]]

,ITEM_ID,title,genre
318,321,Strawberry and Chocolate (Fresa y chocolate) (...,Drama
1358,1379,Young Guns II (1990),Action|Comedy|Western
1213,1232,Stalker (1979),Mystery|Sci-Fi
608,612,"Pallbearer, The (1996)",Comedy
50,51,Guardian Angel (1994),Action|Drama|Thriller
296,299,Priest (1994),Drama
1089,1105,Children of the Corn IV: The Gathering (1996),Horror
1704,1757,Duoluo tianshi (1995),Drama
16,17,Sense and Sensibility (1995),Drama|Romance
36,37,Across the Sea of Time (1995),Documentary


In [28]:
movies.loc[40:100]

,ITEM_ID,title,genre
40,42,Dead Presidents (1995),Action|Crime|Drama
41,43,Restoration (1995),Drama
42,44,Mortal Kombat (1995),Action|Adventure|Fantasy
43,45,To Die For (1995),Comedy|Drama|Thriller
44,46,How to Make an American Quilt (1995),Drama|Romance
45,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
46,48,Pocahontas (1995),Animation|Children|Drama|Musical|Romance
47,49,When Night Is Falling (1995),Drama|Romance
48,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
49,52,Mighty Aphrodite (1995),Comedy|Drama|Romance


In [73]:
# Read in CSV file and create simple lookup dictionary, we could use pandas, however this pulls in a huge dependency and we want to keep it simple
moviesDict = {}
movies = csv.DictReader(open("movies.csv"))
for row in movies:
    moviesDict.update({row['ITEM_ID'] : {'id': row['ITEM_ID'], 'title': row['title'], 'genre': row['genre']}})
moviesDict['1']




{'id': '1',
 'title': 'Toy Story (1995)',
 'genre': 'Adventure|Animation|Children|Comedy|Fantasy'}

In [68]:
def searchItemByTitle(movies, title):
    for k in movies:
        if title.lower() in movies[k]['title'].lower():
            return movies[k]
    return None

In [69]:
movieName = 'jum'
searchItemByTitle(moviesDict, movieName)

{'id': '2', 'title': 'Jumanji (1995)', 'genre': 'Adventure|Children|Fantasy'}